In [17]:
s = open("realpython_urls.txt")

In [18]:
s = s.read()

In [19]:
import re

In [20]:
t = re.findall(r'(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+',s)

In [21]:
import pickle
filename = "saved_thing"
outfile = open("clean_real_python.txt","wb")


In [22]:
import re
s=open("dev_to_sitemap.xml")
s = s.read()
t = re.findall(r'(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+',s)
outfile = open("clean_dev_to_sitemap","wb")
pickle.dump(t, outfile)
outfile.close()

In [23]:
def get_urls(string):
    if "dev.to" in string:
        if "hi-im" not in string:
            return string
tt = [get_urls(i) for i in t]
tt = [i for i in tt if i is not None]
outfile = open("clean_dev_to_sitemap","wb")
pickle.dump(tt, outfile)
outfile.close()

# possible keywords
django, javascript, lisp, emacs, clojure, clojurescript, react, swift, vim, go, c++, linux, flask, angular, html, css, bootstrap, json, postgres, ruby, rails, materialui, git, ci, travis-ci, circleci, aws, mongodb, c, interviews, vr, rust, elm, tdd, oop, functional-programming, algorithms, typescript, android, node, kotlin, oauth, 

In [8]:
len(tt)

45075

# Scrape

In [24]:
import pickle
filename = "clean_dev_to_sitemap"
infile = open(filename, "rb")
urls = pickle.load(infile)
infile.close()

In [25]:
del urls[0]

In [26]:
import requests
from bs4 import BeautifulSoup

In [2]:
a = requests.get("https://dev.to/googlecloud/running-multiple-versions-of-python-in-cloud-run-4hl8")

In [3]:
soup = BeautifulSoup(a.text,"html.parser")

In [4]:
l = soup.find_all("p")

In [5]:
l =" ".join([i.text for i in l])

In [6]:
code = soup.find_all("code")

In [7]:
" ".join([i.text for i in code])

'$ python37 recommendations.py\n  File "recommendations.py", line 42\n    while (chunk := stream.read(256)) != \'\':\n                 ^\nSyntaxError: invalid syntax\n FROM python:2.7\n...\n FROM python:3.8\n...\n'

In [29]:
import sqlite3

In [30]:
with sqlite3.connect("inbrain.db") as conn:
    conn.execute("CREATE TABLE scraped_pages (url text PRIMARY KEY, page_text text, page_code text )")

In [50]:
def scrape_pages(url, results):
    r = requests.get(url).text
    soup = BeautifulSoup(r, "html.parser")
    page_text = soup.find_all("p")
    page_text = " ".join([i.text for i in page_text])
    page_code = soup.find_all("c")
    page_code = " ".join([i.text for i in page_code])
    results.append( (url,page_text,page_code)  )

In [51]:
import asyncio
import requests
import nest_asyncio
from concurrent.futures import ThreadPoolExecutor

nest_asyncio.apply()


async def create_scrape_loop(
    iterable, individual_scrape_function, num_workers=40, *args
):
    """This function is responsible for organizing the asyncrhonous pieces.
    It creates a session, uses a list comprehension to create the tasks,
    and gathers the tasks.
    Parameters
    ----------
    iterable : list, default = None
        This a list of things to repeat with the individual_scrape_function.
        It can be, for example, a list of urls or page numbers.
    individual_scrape_function : function, default = None
        A function designed for scraping a single thing, usually a page or api request.
        Takes an element from the iterable and uses that to do multiple operations.
    num_workers : <class 'int'>, default = 40
        #TODO description
    Returns
    -------
    """
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        with requests.Session() as session:
            loop = asyncio.get_event_loop()
            tasks = [
                loop.run_in_executor(executor, individual_scrape_function, *(i, *args))
                for i in iterable
            ]
            for response in await asyncio.gather(*tasks):
                pass


def async_web_scrape(iterable, individual_scrape_function, num_workers, *args):
    """This function takes a function designed to scrape a single thing,
    i.e. a web page or api request. The individual_scrape_function gets
    passed to the the create_loop_function with the iterable and other args.
    Parameters
    ----------
    iterable : list, default = None
        This a list of things to repeat with the individual_scrape_function.
        It can be, for example, a list of urls or page numbers.
    individual_scrape_function : function, default = None
        A function designed for scraping a single thing, usually a page or api request.
        Takes an element from the iterable and uses that to do multiple operations.
    num_workers : <class 'int'>, default = 40
        The number of workers for the thread. This is passed directly to the 
        create_scrape_loop function.
    *args : these are the parameters of the individual scrape function.
    Returns
    -------
        Nothing. Unfortunately, this happens to use side effects. 
        Maybe in the future this can be changed.
    """
    future = asyncio.ensure_future(
        create_scrape_loop(iterable, individual_scrape_function, num_workers, *args)
    )
    loop = asyncio.get_event_loop()
    loop.run_until_complete(future)

In [52]:
for i in range(3,47):
    print(i)
    search = urls[i*1_000:(i+1)*1_000]
    results = []
    async_web_scrape(search,scrape_pages,40,results)
    
    
    with sqlite3.connect("inbrain.db") as conn:
        conn.executemany("INSERT INTO scraped_pages VALUES (?,?,?)",results)

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46


In [53]:
with sqlite3.connect("inbrain.db") as conn:
    count = conn.execute("SELECT COUNT(url) from scraped_pages").fetchone()
    print(count)
    

(45074,)
